<a href="https://colab.research.google.com/github/spe301/AI-generated-AI/blob/main/AI_generated_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Potosnail==0.2.1

In [12]:
import pandas as pd
import numpy as np
from potosnail import *

ml = MachineLearning()
dl = DeepLearning()
dh = DataHelper()
ev = Evaluater()
al = Algorithms()
wr = Wrappers()
st = Stats()

def SuperBear(df):
  dh = DataHelper()
  '''gets the nlp results dataset ready for modeling'''
  df['regularizer'] = df['regularizer'].fillna('None')
  df['stacking'] = df['stacking'].astype(int)
  df['dropout'] = df['dropout'].astype(int)
  df['bidirectional'] = df['bidirectional'].astype(int)
  act = dh.OHE(df['activation'])
  reg = dh.OHE(df['regularizer'])
  opt = dh.OHE(df['optimizer'])
  method = dh.OHE(df['method'])
  df = df.drop(['activation', 'regularizer', 'optimizer', 'method'], axis='columns')
  df = pd.concat([df, act, reg, opt, method], axis='columns')
  df['val_loss'] = df['val_loss'].fillna(max(df['val_loss']))
  df['loss'] = df['loss'].fillna(max(df['loss']))
  return df

In [13]:
df1 = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/NLP8.csv').drop(['Unnamed: 0'], axis='columns')
nlp = SuperBear(df1)

# EDA Questions

1. is the difference in validation loss between 9,000 point datasets and 360 point datasets statistically signifigant?
2. is the difference in 'difference of validation loss and loss' between dropout and no dropout statistically signifigant?
3. Which model had the highest epochs and how many standard deviations away were it's accuracies from average (z score)

In [16]:
from scipy.stats import ttest_ind

s1 = nlp.loc[nlp['len_dataset'] == 360]['val_loss'].sample(100)
s2 = nlp.loc[nlp['len_dataset'] != 360]['val_loss'].sample(100)
ttest_ind(s1, s2)

Ttest_indResult(statistic=8.087492061735881, pvalue=5.982451503934567e-14)

validation loss is signifigantly lower with larger dataset

conclusion: Deep NLP works better for larger datasets with thousands of datapoints.

In [26]:
ldd = nlp.loc[nlp['dropout'] == 1]['val_loss'] - nlp.loc[nlp['dropout'] == 1]['loss']
s1 = ldd.sample(100)
ldno = nlp.loc[nlp['dropout'] == 0]['val_loss'] - nlp.loc[nlp['dropout'] == 0]['loss']
s2 = ldno.sample(100)
ttest_ind(s1, s2)

Ttest_indResult(statistic=1.2000054035113255, pvalue=0.23157109438574167)

We can accept the null hypothesis that dropout does not cause a signifigant change in loss

conclusion: adding a dropout layer is unlikley to decrease overfitting.

In [36]:
acc50 = np.mean(list(nlp.loc[nlp['epochs'] == max(nlp['epochs'])]['accuracy']))
val50 = np.mean(list(nlp.loc[nlp['epochs'] == max(nlp['epochs'])]['val_accuracy']))

st.Z(nlp['accuracy'], acc50), st.Z(nlp['val_accuracy'], val50)

(-1.8442059083699316, -0.43293229648074893)

In [38]:
val50

0.4987113395917047

Models that were trained for 50 epochs had lower accuracies than average but were also less overfit.

Conclusion: overfitting seems to be a major problem, setting epochs higher while figuring out how to increase accuracy may be well worth it.

In [39]:
np.mean(nlp['epochs'])

12.6425

In [57]:
kpi_list = ['accuracy', 'loss', 'val_accuracy', 'val_loss']
kpi = nlp[kpi_list]

scores = []
for i in range(len(nlp)):
  ts = (1 - (kpi['loss'][i] / max(kpi['loss'])) + kpi['accuracy'][i])/2
  vs = (1 - (kpi['val_loss'][i] / max(kpi['val_loss'])) + kpi['val_accuracy'][i])/2
  score = (ts+vs) - abs(ts-vs)
  scores.append(score)

nlp2 = nlp.drop(kpi_list, axis='columns')
nlp2['quality'] = scores
nlp2

,output_dim,embedding,nodes,stacking,dropout,bidirectional,epochs,batch_size,len_dataset,n_features,dominant_class,relu,tanh,L1,L2,None,adam,rmsprop,sgd,GRU,LSTM,quality
0,3,12300,64,0,0,0,7,32,8936,17,0.602954,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.338706
1,3,12300,64,1,0,0,7,32,8936,17,0.602954,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.361174
2,3,12300,64,0,0,0,7,32,8936,17,0.602954,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.361862
3,3,12300,64,1,0,0,6,32,8936,17,0.602954,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.387474
4,2,45589,64,0,0,0,6,32,360,1724,0.500000,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.490305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,3,12300,256,0,1,1,6,32,8936,17,0.602954,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.405428
1996,3,12300,256,1,0,1,6,32,8936,17,0.602954,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.395079
1997,3,12300,256,1,0,1,7,32,8936,17,0.602954,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.341054
1998,3,12300,256,1,1,1,6,32,8936,17,0.602954,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.342763


In [58]:
train, val = dh.HoldOut(nlp2)

In [60]:
wr.WrapML(train, 'quality', 'regression', quiet=False)

(200, 22)